In [ ]:
using LibGit2
using Dates
using JSON

In [ ]:
function GetAllCommits(master_repo)
    return LibGit2.with(LibGit2.GitRevWalker(master_repo)) do walker
        LibGit2.map((hash, repo)->LibGit2.GitCommit(repo, hash), walker, by=LibGit2.Consts.SORT_TIME)
    end
end

In [ ]:
function GetTaggedCommits(repo)
    taggedCommits = Dict{String, LibGit2.GitCommit}()
    tagStrings = LibGit2.tag_list(repo)
    for tagString in tagStrings
        oid = LibGit2.GitObject(repo, tagString)
        commit = LibGit2.peel(LibGit2.GitCommit, oid)
        taggedCommits[tagString] = commit
    end
    return taggedCommits
end

In [ ]:
function GetCommitKey(repo)
    commitHash = string(LibGit2.GitHash(commit))
    return joinpath(userName, repoName, commitHash)
end

In [ ]:
function GetJSONOfAllCommits(rootFolder::String)
    
    # username/reponame/commitHash --> ["commitAttribute"] = commitValue
    dictionaryOfAllCommits = Dict{String, Dict{String,Any}}()
    
    userNames = readdir(rootFolder)
    for userName in userNames
        
        repoNames = readdir(joinpath(rootFolder, userName))
        for repoName in repoNames
            
            repoPath = joinpath(rootFolder, userName, repoName, "main")
            
            # Some folders are empty
            if ! isdir(repoPath)
                continue
            end
            
            repo = LibGit2.GitRepo(repoPath)

            commits = GetAllCommits(repo)
            for commit in commits
                commitKey = GetCommitKey(repo)
                dictionaryOfAllCommits[commitKey] = Dict{String, Any}()
                commitAttributes = dictionaryOfAllCommits[commitKey]
                commitAttributes["authorName"] = LibGit2.author(commit).name
                commitAttributes["authorEmail"] = LibGit2.author(commit).email
                commitAttributes["committerName"] = LibGit2.committer(commit).name
                commitAttributes["committerEmail"] = LibGit2.committer(commit).email
                commitAttributes["dateTime"] = string(Dates.unix2datetime(LibGit2.author(commit).time))
                commitAttributes["message"] = LibGit2.message(commit)
                commitAttributes["isTagged"] = 0
                commitAttributes["tagName"] = ""
            end
            
            taggedCommits = GetTaggedCommits(repo)
            for (tag, commit) in taggedCommits
                commitKey = GetCommitKey(repo)
                if ! haskey(dictionaryOfAllCommits, commitKey)
                    println(commitHash)
                else 
                    println("success")
                    commitAttributes = dictionaryOfAllCommits[commitKey]
                    commitAttributes["isTagged"] = 1
                    commitAttributes["tagName"] = tag
                end
            end
        end
    end
    return dictionaryOfAllCommits
end

In [ ]:
# cd("..")
pwd()

In [ ]:
providerPath = joinpath(pwd(), "TfCloneRepos_providers")
providerCommitJson = GetJSONOfAllCommits(providerPath)

In [ ]:
modulePath = joinpath(pwd(), "TfCloneRepos_modules")
moduleCommitJson = GetJSONOfAllCommits(modulePath)

In [ ]:
function SaveJSONToFile(objectType::String, dictionaryOfAllCommits::Dict{String,Dict{String,Any}})
    
    # Create a subfolder of this folder called "terraform-registry"
    subfolderpath = joinpath(pwd(), "TfRepoCommitLog")
    if !isdir(subfolderpath)
        mkdir(subfolderpath)
    end

    # Name the file "(providers || modules)_(today's date).json"
    filename = 
        objectType * "_" * 
        string(Dates.today()) * 
        ".json"
    
    filepath = joinpath(subfolderpath, filename)

    # Write the JSON to the file
    file = open(filepath, "w")
    JSON.print(file, dictionaryOfAllCommits)
    close(file)
end

In [ ]:
SaveJSONToFile("providers", providerCommitJson)

In [ ]:
SaveJSONToFile("providers", providerCommitJson)# Do statistics on commits?

In [ ]:
# Do statistics on commits?